In [107]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import camb
from camb import model, initialpower
from camb.sources import GaussianSourceWindow, SplinedSourceWindow
import numpy as np
import sympy as sy
from sympy import symbols
from scipy import interpolate
from scipy import optimize
import matplotlib
from matplotlib import pyplot as plt
import ngfunction

In [108]:
##PARAMETRI INIZIALI

pars = camb.CAMBparams()  #0.022 0.122 67.5
pars.set_cosmology(H0=73, ombh2=0.023, omch2=0.111)   #H0= hubble parameter today in km/s    ombh2=physical density in baryons omch2=physical density in cold dark matter
#trovo densità diverse...
pars.InitPower.set_params(ns=0.965)  #ns = scalar spectra index (inflaction)


class: <InitialPowerLaw>
 tensor_parameterization = tensor_param_rpivot
 ns = 0.965
 nrun = 0.0
 nrunrun = 0.0
 nt = -0.0
 ntrun = -0.0
 r = 0.0
 pivot_scalar = 0.05
 pivot_tensor = 0.05
 As = 2e-09
 At = 1.0
 

In [109]:
zetas= np.array([0.7,1.4,2])
pars.set_matter_power(redshifts=zetas, kmax=1.0)  
pars.NonLinear = model.NonLinear_none
results = camb.get_results(pars)   

Note: redshifts have been re-sorted (earliest first)


In [110]:
kh, z, pk = results.get_matter_power_spectrum(minkh=2e-2, maxkh=1e-1, npoints = 100) #nota, le z vengono ordinate, ricavo spettro di potenza
z

[0.7, 1.4, 2.0]

In [111]:
H0 = results.hubble_parameter(0)    #parametri di hubble
h = H0/100
def H(z):
    return results.hubble_parameter(z)

def dist(z):   #funzione per trasformare il redshift in distanza
    return results.comoving_radial_distance(z) #coefficiente per trasformare il redshift in un raggio, c/H0 = [km/s  / (km/s)/Mpc] = [Mpc], z è adimensionale

kh = kh/h  #H0 = 70km/(s*Mpc) h=0.7 ma non era senza unità di misura?
pk = pk*h**3

In [112]:
z = np.array(z)

In [113]:
om = results.get_Omega('cdm',0) + results.get_Omega('baryon',0)    #densità di materia nell'universo
omz= om*(np.ones(len(z))+z)**3/(H(z)/H0)**2 #densità di materia a diverso z
omz

array([0.62105839, 0.81937971, 0.89630525])

Derivata composta della distanza cìdi luminosità

dr/dz =  c/H

dl = (1+z)*r 

dl'= r + (1+z)*c/H

In [114]:
bias = np.ones(len(z)) + z*0.84   #funzione di bias delle galassie, cercane altre!!

f = (results.get_fsigma8()/results.get_sigma8())
dl = results.luminosity_distance(z[0])
d= results.comoving_radial_distance(z[0])
 

fl= (2*d/(np.ones(len(z))+z))/(np.ones(len(z))*d+(np.ones(len(z))+z)*2.99e5/H(z))

mu = np.arange(-1,1,0.1)      #range di valori dell'angolo
N = len(kh)*len(mu)   #numero di dati simulati


In [115]:
#Set di dati simulati spettro di potenza delle galassie

Pgg = np.empty((len(kh),len(mu),len(zetas)))
for i in range (len(kh)):
    for j in range (len(mu)):
        for m in range (len(zetas)):
            Pgg[i][j][m] = float((bias[m] + fl[m] *f[m] * mu[j]**2)**2*pk[m,i])


In [116]:
err_Pgg = np.ones((len(kh),len(mu),len(zetas)))

In [117]:
gammas= np.arange(0.4,0.6,0.005)   #calcolo il chi quadro per un certo set di valori di gamma

for k in range(len(gammas)):
    gammas[k] = round (gammas[k],3)

chi2 = np.zeros((len(gammas),len(zetas)))


for k,gamma in enumerate(gammas):
    for i in range (len(kh)):
        for j in range (len(mu)):
            for m in range (len(zetas)):
                chi2[k,m]= chi2[k,m]+(float((bias[m] + (omz[m]**gamma)*fl[m] * (mu[j])**2)**2*pk[m,i]) - Pgg[i][j][m])**2/err_Pgg[i][j][m]**2
        



In [118]:
#INTERPOLAZIONE FUNZIONE CHI^2

interpol1 =  interpolate.interp1d(gammas,chi2[:,0],kind='cubic')
interpol2 =  interpolate.interp1d(gammas,chi2[:,1],kind='cubic')
interpol3 =  interpolate.interp1d(gammas,chi2[:,2],kind='cubic')
#plt.plot(gammas,chi2,'-', gammas, interpol(gammas), '--')
#plt.legend(['data', 'cubic'])
#plt.xlabel(' gamma ')
#plt.ylabel('Chi^2')


#plt.axis([0.54,0.56,0,3])

In [119]:
type(interpol1)
y1=lambda x: interpol1(x)

type(interpol2)
y2=lambda x: interpol2(x)

type(interpol3)
y3=lambda x: interpol3(x)


In [120]:

minimo1 = optimize.least_squares(y1,0.5)
min1= float(minimo1.x)
chi2min1= y1(min1)


minimo2 = optimize.least_squares(y2,0.5)
min2= float(minimo2.x)
chi2min2= y2(min2)

minimo3 = optimize.least_squares(y3,0.5)
min3= float(minimo3.x)
chi2min3= y3(min3)


ValueError: A value in x_new is below the interpolation range.

In [ ]:
y1_= lambda x: interpol1(x) - 1 -chi2min1
intersec1a = optimize.brentq(y1_,0.45,min1)    #trova le soluzioni in un intervallo fissato
intersec1b = optimize.brentq(y1_,min1,0.58)
print(intersec1a,intersec1b)

y2_= lambda x: interpol2(x) - 1 -chi2min2
intersec2a = optimize.brentq(y2_,0.45,min2)    #trova le soluzioni in un intervallo fissato
intersec2b = optimize.brentq(y2_,min2,0.58)
print(intersec2a,intersec2b)

y3_= lambda x: interpol3(x) - 1 -chi2min3
intersec3a = optimize.brentq(y3_,0.45,min3)    #trova le soluzioni in un intervallo fissato
intersec3b = optimize.brentq(y3_,min3,0.58)
print(intersec3a,intersec3b)

0.5397236024635194 0.5397827850608893


In [ ]:
sigma = np.zeros(len(zetas))

In [ ]:
err1a=(np.abs(min1-intersec1a))
err1b=(np.abs(min1-intersec1b))
sigma[0] = round(err1a,3)

err2a=(np.abs(min2-intersec2a))
err2b=(np.abs(min2-intersec2b))
sigma[1] = round(err2a,3)

err3a=(np.abs(min3-intersec3a))
err3b=(np.abs(min3-intersec3b))
sigma[0] = round(err3a,3)



2.909568274500618e-05 3.0086914624916083e-05
0.0


In [ ]:
print("Il valore di gamma trovat è: ", round(min1,3) ," +- ", sigma)

Il valore di gamma trovat è:  0.54  +-  0.0
